### Comparison with wire-scanner measurements

Load the model Twiss parameters at each node.

In [ ]:
model_twiss = pd.read_table(join(folder, 'model_twiss.dat'), sep=' ')
model_twiss

Load the reconstructed beam moments at each node. These were reconstructed from wire-scanner measurements. We should ignore everything downstream of `RTBT_Diag:QH18` because the magnets downstream were changed during the target scan.

In [ ]:
rec_moments = pd.read_table(join(folder, 'rec_moments.dat'), sep=' ')
rec_moments

In [ ]:
rec_stats = ba.BeamStats()
rec_stats.read_moments(rec_moments.iloc[:, 2:].values)
rec_stats.twiss2D

In [ ]:
ws_ids = ['RTBT_Diag:WS20', 'RTBT_Diag:WS21', 'RTBT_Diag:WS23', 'RTBT_Diag:WS24']
ws_positions = []
for ws_id in ws_ids:
    ws_position = float(rec_moments[rec_moments['node_id'] == ws_id]['position']) 
    ws_positions.append(ws_position)

In [ ]:
fig, axes = pplt.subplots(nrows=4, figsize=(3, 5), spany=False)
colors = pplt.Cycle('538').by_key()['color']
plot_kws = dict(marker='.', ms=0)
axes[0].plot(rec_moments['position'], rec_stats.corr['xy'], color=colors[0], **plot_kws)
axes[1].plot(rec_moments['position'], rec_stats.corr['xyp'], color=colors[1], **plot_kws)
axes[2].plot(rec_moments['position'], rec_stats.corr['yxp'], color=colors[2], **plot_kws)
axes[3].plot(rec_moments['position'], rec_stats.corr['xpyp'], color=colors[3], **plot_kws)
axes.format(xlabel='position', ylim=(-1.0, 1.0), suptitle='Cross-plane correlation coefficients.')
axes[0].set_ylabel("x-y")
axes[1].set_ylabel("x-y'")
axes[2].set_ylabel("y-x'")
axes[3].set_ylabel("x'-y'")
for ax in axes:
    for ws_position in ws_positions:
        ax.axvline(ws_position, color='black', alpha=0.2, ls='--', lw=0.5)
plt.savefig('_output/figures/corr_coeff.png', **savefig_kws)
plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(6.5, 2.5))
rec_kws = dict(ls='--', lw=0.75)
model_kws = dict(marker='.', ms=3, lw=1.25)
colors = myplt.DEFAULT_COLORCYCLE
g1 = ax.plot(model_twiss['position'], model_twiss['beta_x'], color=colors[0], **model_kws)
g2 = ax.plot(model_twiss['position'], rec_stats.twiss2D['beta_x'], color=colors[0], **rec_kws)
g3 = ax.plot(model_twiss['position'], model_twiss['beta_y'], color=colors[1], **model_kws)
g4 = ax.plot(model_twiss['position'], rec_stats.twiss2D['beta_y'], color=colors[1], **rec_kws)
ax.legend(handles=[g1, g2, g3, g4], 
          labels=[r'$\beta_x$ (model)', r'$\beta_x$ (rec)',
                  r'$\beta_y$ (model)', r'$\beta_y$ (rec)'], 
          ncols=2, loc='upper left')
ax.format(xlabel='Position [m]', ylabel='[m/rad]')
ax.format(ylim=(0., 120.))
plt.savefig('_output/figures/rec_betas.png', **savefig_kws)
plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(6.5, 2.5))
rec_kws = dict(ls='--', lw=0.75)
colors = myplt.DEFAULT_COLORCYCLE
g1 = ax.plot(model_twiss['position'], model_twiss['alpha_x'], color=colors[0], **model_kws)
g2 = ax.plot(rec_moments['position'], rec_stats.twiss2D['alpha_x'], color=colors[0], **rec_kws)
g3 = ax.plot(model_twiss['position'], model_twiss['alpha_y'], color=colors[1], **model_kws)
g4 = ax.plot(rec_moments['position'], rec_stats.twiss2D['alpha_y'], color=colors[1], **rec_kws)
ax.legend(handles=[g1, g2, g3, g4], 
          labels=[r'$\alpha_x$ (model)', r'$\alpha_x$ (rec)',
                  r'$\alpha_y$ (model)', r'$\alpha_y$ (rec)'], 
          ncols=2, loc='upper left')
ax.format(xlabel='Position [m]', ylabel='[rad]')
plt.savefig('_output/figures/rec_alphas.png', **savefig_kws)
plt.show()

The measured Twiss parameters seem to agree reasonably well with the model at `RTBT_Diag:BPM16`. We will use this as our reconstruction location; although the method functions the same for any reconstruction location, choosing a point where the beam is matched will make the projection angles close to the phase advances in normalized phase space. Also, the reconstructed distribution will be more round in normalized phase space.

In [ ]:
rec_node_id = 'RTBT_Diag:BPM16'

idx = model_twiss['node_id'] == rec_node_id
rec_node_id, position, alpha_x, alpha_y, beta_x, beta_y = model_twiss.loc[idx].values[0]
print('rec_node_id = {} [m]'.format(rec_node_id))
print('position = {:.2f}'.format(position))
print('alpha_x = {:.2f} [rad]'.format(alpha_x))
print('alpha_y = {:.2f} [rad]'.format(alpha_y))
print('beta_x = {:.2f} [m/rad]'.format(beta_x))
print('beta_y = {:.2f} [m/rad]'.format(beta_y))

V = V_matrix_4x4_uncoupled(alpha_x, alpha_y, beta_x, beta_y)
Vx = V[:2, :2]
Vy = V[2:, 2:]
print('V =')
print(V)

### Transfer matrices 

Load the transfer matrices from this node to the target for each optics setting.

In [ ]:
tmat_filenames = [f for f in filenames if 'tmat' in f]
tmat_filenames = sorted(tmat_filenames, key=lambda f: float(f.split('.')[0].split('_')[-1]))
tmats = []
for filename in tmat_filenames:
    file = open(filename, 'r')
    lines = [line.rstrip() for line in file]
    for line in lines[1:]:
        tokens = line.rstrip().split()
        node_id, tmat_elems = tokens[0], [float(token) for token in tokens[1:]]
        if node_id == rec_node_id:
            tmats.append(np.array(tmat_elems).reshape((4, 4)))
    file.close()
    
tmats_x = [tmat[:2, :2] for tmat in tmats]
tmats_y = [tmat[2:, 2:] for tmat in tmats]

tmats_norm = [np.matmul(M, V) for M in tmats]
tmats_x_norm = [tmat[:2, :2] for tmat in tmats_norm]
tmats_y_norm = [tmat[2:, 2:] for tmat in tmats_norm]